# Explorative Datenanalyse
Im Folgenden werden die gemessenen Index-Werte der Brokkoli analysiert. Im Vorfeld wurden diese bereits innerhalb der Voronoi-Zellen segmentiert, das heisst Es werden nur noch die Pixel gewertet, welche zum Brokkoli gehören, ohne die Erde darum zu berücksichtigen.

In [ ]:
%reset

# Datenabfrage von Server-DB
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

class font:
    BOLD = '\033[1m'
    END = '\033[0m'
    UNDERLINE = '\033[4m'

# Liste der Daten, die berücksichtigt werden
dates = ['2019-04-18', '2019-04-25', '2019-05-01', '2019-05-09', '2019-05-16', '2019-05-24', '2019-06-04','2019-06-13', 
         '2019-06-18']

# Angeben, ob Blacklist einbezogen wird
useBlacklist = True

# Verbindungsaufbau
server = 'deepbroccoliserver.database.windows.net'
database = 'DeepBroccoliDatabase'
username = 'ntb'
password = 'brokkoli_2019'
driver= '{SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)

# Abfrage definieren und ausführen: Alle Brokkoli mit allen dazugehörigen Messwerten 
query = '''
select dbo.broccoli.id, dbo.broccoli.cropWeight, dbo.broccoli.cropMissing, dbo.broccoli.cropOverripe, 
dbo.broccoli.cropRudimentary, dbo.broccoli.cropRotten, dbo.broccoli.cropUnripe, dbo.broccoli.cropNoBlossom,  
dbo.broccolivalues.timestamp, dbo.broccoli.lat, dbo.broccoli.long, dbo.broccolivalues.pixelCount, dbo.broccolivalues.maxNDVI, 
dbo.broccolivalues.minNDVI, dbo.broccolivalues.meanNDVI, dbo.broccolivalues.medianNDVI, 
dbo.broccolivalues.NDVI_15_QUANTILE, dbo.broccolivalues.NDVI_25_QUANTILE, dbo.broccolivalues.NDVI_75_QUANTILE, 
dbo.broccolivalues.NDVI_85_QUANTILE,
dbo.broccolivalues.maxNDRE, dbo.broccolivalues.minNDRE, dbo.broccolivalues.meanNDRE, dbo.broccolivalues.medianNDRE,
dbo.broccolivalues.NDRE_15_QUANTILE, dbo.broccolivalues.NDRE_25_QUANTILE, dbo.broccolivalues.NDRE_75_QUANTILE,
dbo.broccolivalues.NDRE_85_QUANTILE
from dbo.broccoli inner join dbo.broccolivalues on dbo.broccoli.id = dbo.broccolivalues.id'''

# Blacklist einbeziehen: Brokkoli-IDs in Blacklist werden gefiltert
if useBlacklist:
    query = query + ''' where dbo.broccoli.id not in (select broccoli_id from dbo.broccoliBlacklist)'''

broccoli_data = pd.read_sql_query(query, cnxn)
cnxn.close()

# Löschen von Datensätzen mit NaN-Werten (Ist der Fall falls pixelCount == 0)
#print()
#print("Anzahl Datensätze: " + str(len(broccoli_data)))

#broccoli_data.dropna(inplace=True)
#print("Anzahl Datensätze ohne NaN: " + str(len(broccoli_data)))
#print()

# Describe: Erste Übersicht
for date in dates:    
    broccoli_data_byDate = broccoli_data[broccoli_data.timestamp == date]
    print(font.BOLD + font.UNDERLINE + date + font.END)
    display(broccoli_data_byDate.describe())
    print()

In [ ]:
# True/False Erntelabels zu 0/1
broccoli_data.cropMissing = broccoli_data.cropMissing.astype(int)
broccoli_data.cropOverripe = broccoli_data.cropOverripe.astype(int)
broccoli_data.cropRudimentary = broccoli_data.cropRudimentary.astype(int)
broccoli_data.cropRotten = broccoli_data.cropRotten.astype(int)
broccoli_data.cropUnripe = broccoli_data.cropUnripe.astype(int)
broccoli_data.cropNoBlossom = broccoli_data.cropNoBlossom.astype(int)


## Analyse der Pixel-Anzahl (pixelCount)
Als erstes werden die Anzahl Pixel, die nach der Segmentierung eine Brokkoli-Pflanze darstellen, analysiert. Hier wird versucht, inkorrekt oders seltsam segmentierte Daten zu finden.

In [ ]:
# Boxplot der Pixelanzahl gruppiert nach Datum
broccoli_data.boxplot(by="timestamp", column=['pixelCount'], figsize=(10,8))
plt.xticks(rotation=45)

### Zwischenfazit Pixel-Anzahl Boxplot
Es ist komisch, dass vom 13.06.19 auf den 18.06.19 der pixelCount niedriger wird.<br>
ToDo: Die Segmentierung muss geprüft und allenfalls überarbeitet werden.

In [ ]:
# Analyse pixelCount: Datensätze in Bereich ausserhalb bestimmter Quantile analysieren
# Erstellung von CSV und HTML Export zur manuellen Analyse und Erstellung eine Blacklist mit inkorrekt segmentierten Datensätzen

pixelCountNotable = pd.DataFrame()
for date in dates:    
    # Listen mit Pflanzen mit pixelCount > [quantile_Lower]-Quantil bzw. < [quantile_Upper]-Quantil erstellen
    broccoli_data_byDate = broccoli_data[broccoli_data.timestamp == date]
    
    quantile_Lower = 0.03
    quantile_Upper = 0.97
    
    underLowerQuant = broccoli_data_byDate[broccoli_data_byDate.pixelCount < 
                                        broccoli_data_byDate['pixelCount'].quantile(quantile_Lower)][['id', 'pixelCount']]
    overUpperQuant = broccoli_data_byDate[broccoli_data_byDate.pixelCount > 
                                       broccoli_data_byDate['pixelCount'].quantile(quantile_Upper)][['id', 'pixelCount']]
    
    # Liste aller gefilterter IDs nach Datum erstellen
    underLowerQuant[date + "_Under"] = 1
    overUpperQuant[date + "_Over"] = 1
    underLowerQuant.set_index('id', inplace=True)
    overUpperQuant.set_index('id', inplace=True)
    pixelCountNotable = pd.concat([pixelCountNotable, underLowerQuant[date + "_Under"]], axis=1)
    pixelCountNotable = pd.concat([pixelCountNotable, overUpperQuant[date + "_Over"]], axis=1)    
    underLowerQuant.drop([date + "_Under"], axis=1, inplace=True)
    overUpperQuant.drop([date + "_Over"], axis=1, inplace=True)
    underLowerQuant.reset_index(inplace=True)
    overUpperQuant.reset_index(inplace=True)
    
    # CSV mit IDs der Pflanzen mit pixelCount < [quantile_Lower]-Quantil oder > [quantile_Upper]-Quantil erstellen
    underLowerQuant['toBlacklist']= 0
    overUpperQuant['toBlacklist']= 0
    np.savetxt("pixelCount_notableIDs_" + date + ".csv", pd.concat([underLowerQuant, overUpperQuant]), 
               header="id;pixelCount;toBlacklist", delimiter=";", fmt="%d", comments='')    
    
    # HTML mit den Segmentierungs-Bildern erstellen
    strHtml = ""
    for index, row in pd.concat([underLowerQuant, overUpperQuant]).iterrows():
        strId = str(row["id"])
        strHtml += "<p>#" + strId + " ; pixelCount = " + str(row["pixelCount"]) + "</p>\n"
        strHtml += "<img src=\"\\\\fs004\\ice\\Lehre\\Bachelorarbeiten\\2019\\Pflanzen\\Drohnenaufnahmen\\" \
        + (str(date)).replace('-', '') + "\\report\\images\\" + strId + "\\ndvi_ndre_cutout_comparision_" + strId + ".png\">\n"
       
    
    html = open("pixelCount_NotableImages_" + date + ".html", 'w')
    html.write(strHtml)
    html.close()
    
    # Ausgabe der Anzahlen
    nofUnderLowerQuant = len(underLowerQuant)
    nofOverUpperQuant = len(overUpperQuant)
    
    print(date + ":: <" + str(quantile_Lower*100) + "%; " + str(nofUnderLowerQuant) + " | >" + str(quantile_Upper*100) + "%; "
          + str(nofOverUpperQuant))
    
# Ausgabe Liste aller gefilterter IDs nach Datum
pixelCountNotable.fillna(value=0, inplace=True)
pixelCountNotable.to_csv("pixelCount_Notable_PerDate.csv", sep=';')

### Zwischenfazit pixelCount, Stand 21.05.2019
Da im Boxplot ersichtlich ist, dass es einige Ausreisser gibt, werden die Datensätze mit pixelCount ausserhalb der 2%- bzw. 98%-Quantile genauer untersucht (Muss allenfalls auf z.B. 5%, 95% ausgeweitet werden). Dazu werden je Datum eine CSV- und eine HTML-Datei erstellt, mit den IDs, dem pixelCount und den Vergleichsbildern der Segmentierung (im HTML). Diese müssen manuell kontrolliert werden, danach können Datensätze entweder gelöscht oder angepasst werden.
Im Vergleich zu den alten Daten (nicht mehr ersichtlich da überschrieben), gibt es aber vorallem nach oben, über der 75%-Quantils-Grenze, weniger krasse Ausreisser.

## Analyse der NDVI und NDRE Messwerte
Im Folgenden werden die aggregierten NDVI und NDRE Werte der segmentierten Bilder analysiert. Es werden Ausreisser und auffällige Messwerte gesucht, zeitliche Verläufe der Werte, und Abhängigkeiten zwischen den Indizes. Ein Plot mit y-Achse = NDRE und x-Achse = NDVI würde die Grundlage für die Berechnung des CCCI bilden.

## NDVI

In [ ]:
# Boxplot der NDVI-Werte gruppiert nach Datum
columns = ['maxNDVI', 'NDVI_85_QUANTILE', 'minNDVI', 'NDVI_15_QUANTILE', 'meanNDVI', 'medianNDVI']
ax = broccoli_data.boxplot(by="timestamp", column=columns, layout=(3,2), figsize=(15,25), rot=90)

### Zwischenfazit NDVI, Stand 21.05.2019
Es ist ersichtlich, dass die NDVI-Werte mit der Zeit im Mittel tendenziell steigen.
Statt der Max-/Min-Werte sollten eher die 15%-/85%-Quantile verwenedet werden, um Ausreisser zu vermeiden. Es ist jedoch ersichtlich, dass diese ähnlich oder fast stärker streuen als die Max-/Min-Werte. 

## NDRE

In [ ]:
# Boxplot der NDRE-Werte gruppiert nach Datum
columns = ['maxNDRE', 'NDRE_85_QUANTILE', 'minNDRE', 'NDRE_15_QUANTILE', 'meanNDRE', 'medianNDRE']
ax = broccoli_data.boxplot(by="timestamp", column=columns, layout=(3,2), figsize=(15,25), rot=90)

### Zwischenfazit NDRE, Stand 21.05.2019
Bei dem NDRE-Messwerten ist ersichtlich, dass die Streuung um den Mittelwert bei den 15%-/85%-Quantilen kleiner ist als bei den Min-/Max-Werten. zudem steigen die NDRE Messwerte nicht kontinuierlich, sondern fallen und steigen wieder. Dass die unteren und oberen Werten ähnlich fallen und steigen, lässt auf eine gewisse Konsistenz in der Datenerfassung schliessen.

## Plot NDVI/NDRE
Für die allfällige Berechnung des CCCI ist ein Scatterplot mit x-Achse = NDVI und y-Achse = NDRE die Voraussetzung. Im Folgenden wird dieser mit den 85%-Quantilen und mit den Mittelwerten der Datensätze erstellt.

In [ ]:
import seaborn as sns 
sns.lmplot(x='NDVI_85_QUANTILE', y='NDRE_85_QUANTILE', data=broccoli_data, hue='timestamp', fit_reg=False)
axes = plt.gca()
axes.set_xlim([0,1])
axes.set_ylim([0,1])
plt.show()

In [ ]:
sns.lmplot(x='maxNDVI', y='maxNDRE', data=broccoli_data, hue='timestamp', fit_reg=False)
axes = plt.gca()
axes.set_xlim([0,1])
axes.set_ylim([0,1])
plt.show()

In [ ]:
sns.lmplot(x='pixelCount', y='meanNDVI', data=broccoli_data, hue='timestamp', fit_reg=False)
axes = plt.gca()
axes.set_ylim([0,1])
plt.show()

In [ ]:
sns.lmplot(x='pixelCount', y='NDRE_85_QUANTILE', data=broccoli_data, hue='timestamp', fit_reg=False)
axes = plt.gca()
axes.set_ylim([0,1])
plt.show()

### Zwischenfazit NDVI/NDRE, Stand 21.05.2019
Wie in den Scatterplots ersichtlich ist, sind die Datenwolken nach Datum einigermassen örtlich gruppiert. Dabei treten jedoch sehr verschiedene NDRE-Werte pro Datum und auch NDVI-Wert auf. Eine lineare Beziehung NDVI->NDRE, wie sie in der Literatur (vergleiche Fachmodul) gezeigt wurde, ist nicht wirklich sichtbar. Die Berechnung des CCCI anhand dieser Daten ist momentan noch nicht vielversprechend, könnte jedoch gewagt werden. Abzuwarten ist, ob die NDRE-Werte mit der Zeit noch ansteigen, da sie mittlerweile eher tiefe Werte annehmen.

## Analyse der Erntedaten
In diesem Schritt werden die Erntedaten, dass heisst Gewicht und Zustand, in Zusammenhang mit den Messdaten (Vegetationsindizes, Wetterdaten) untersucht.<br>
Versucht wird eine Korrelation zwischen den Indizes oder dem Wetter und dem Erntegewicht zu finden. Das Erntegewicht ist hauptsächlich die Response eines potentiellen ML-Modelles, wobei auch noch der Zustand berücksichtigt werden kann. Zustände sind:<br>
* Fehlt: Brokkoli wurde bei der Ernte nicht gefunden
* Überreif
* Verkümmert: Pflanze ist fast gar nicht gewachsen und hat kleine Blätter
* Kein Kopf: Pflanze hat zwar grosse Blätter aber keinen 'Kopf' bzw. keine Blüte
* Faul
* Unreif (alle Brokkoli unter 300g)

### Korrelationen zwischen Vegetationsindizes, Pixel-Anzahl und Erntegewicht
In einem ersten Schritt wird durch verschiedene Plots versucht, eine erhoffte Korrelation der Vegetationsindizes oder der Pixel-Anzahl (gemäss Segmentierung) und dem Erntegewicht zu finden.

#### Untersuchung letzte Messung - Unverrechnete Werte
Da die Pflanzen bei der letzten Messung am 18.06.19, am Tag vor der Ernte, am weitesten entwickelt waren, wird zuerst diese Messung mit den Erntedaten verglichen.

In [ ]:
# Daten mit den Messwerten der letzen Messung filtern
broccoli_data_lastDate = broccoli_data[broccoli_data.timestamp == '2019-06-18']

# Scatterplot NDRE 85%-Quantil - Gewicht
sns.jointplot(x='NDRE_85_QUANTILE', y='cropWeight', data=broccoli_data_lastDate, kind='reg',
             joint_kws={'line_kws':{'color':'cyan'}})

# Horizontale Linie, welche das Minimalgewicht für Reife darstellt
plt.axhline(y=300, color='red')

# Skala NDRE auf [0,1] setzen
axes = plt.gca()
axes.set_xlim([0,1])
plt.show()

In [ ]:
# Scatterplot NDRE 15%-Quantil - Gewicht
sns.jointplot(x='NDRE_15_QUANTILE', y='cropWeight', data=broccoli_data_lastDate, kind='reg',
             joint_kws={'line_kws':{'color':'cyan'}})

# Horizontale Linie, welche das Minimalgewicht für Reife darstellt
plt.axhline(y=300, color='red')

# Skala NDRE auf [0,1] setzen
axes = plt.gca()
axes.set_xlim([0,1])
plt.show()

In [ ]:
# Scatterplot NDVI 85%-Quantil - Gewicht
sns.jointplot(x='NDVI_85_QUANTILE', y='cropWeight', data=broccoli_data_lastDate, kind='reg',
             joint_kws={'line_kws':{'color':'cyan'}})

# Horizontale Linie für minimales Erntegewicht
plt.axhline(y=300, color='red')

# Skala NDVI auf [0,1] setzen
axes = plt.gca()
axes.set_xlim([0,1])
plt.show()

In [ ]:
# Scatterplot Pixel-Anzahl - Erntegewicht
sns.jointplot(x='pixelCount', y='cropWeight', data=broccoli_data_lastDate, kind='reg',
             joint_kws={'line_kws':{'color':'cyan'}})

# Horizontale Linie für minimales Erntegewicht
plt.axhline(y=300, color='red')
plt.show()

#### Untersuchung verrechnete Werte
Da keine ersichtliche Korrelation zwischen NDVI/NDRE/Pixel-Anzahl und Gewicht gefunden wurde, werden in diesem Schritt bestimmte Werte berechnet, die nicht nur Messwerte an einem Datum darstellen. Das kann beispielsweise die Summe des durchschnittlichen NDVI über alle Messdaten sein.

In [ ]:
# Erstellung eines DataFrames zur Untersuchung der Erntedaten mit verrechneten Werten
# broccoli_data_lastDate beinhaltet zu jedem Brokkoli nur ein Datum, das DataFrame enthält zu Beginn nur Brokkoli-IDs 
# und deren Gewicht
broccoli_cropData = broccoli_data_lastDate[['id', 'cropWeight','cropMissing', 'cropOverripe', 
'cropRudimentary', 'cropRotten', 'cropUnripe', 'cropNoBlossom']].copy()

# Berechnung der Differenz des NDRE-85%-Quantils zwischen dem letzen und dem ersten Messdatum für jeden Brokkoli
broccoli_cropData = broccoli_cropData.merge(broccoli_data[broccoli_data.timestamp == '2019-06-18'][['id', 'NDRE_85_QUANTILE']], 
                                            left_on='id', right_on='id')
broccoli_cropData = broccoli_cropData.merge(broccoli_data[broccoli_data.timestamp == '2019-04-18'][['id', 'NDRE_85_QUANTILE']], 
                                            left_on='id', right_on='id', suffixes=('_0618', '_0418'))
broccoli_cropData['NDRE_85_QUANTILE_diff'] = (broccoli_cropData['NDRE_85_QUANTILE_0618'] - 
                                              broccoli_cropData['NDRE_85_QUANTILE_0418'])
broccoli_cropData.drop(['NDRE_85_QUANTILE_0618', 'NDRE_85_QUANTILE_0418'], axis=1, inplace=True)

# Scatterplot Differenz NDRE-85%-Quantil - Gewicht
sns.jointplot(x='NDRE_85_QUANTILE_diff', y='cropWeight', data=broccoli_cropData, kind='reg',
             joint_kws={'line_kws':{'color':'cyan'}})

# Horizontale Linie für minimales Erntegewicht
plt.axhline(y=300, color='red')
plt.show()

In [ ]:
# Berechnung der Summen aller Messwerte für jeden Brokkoli
broccoli_cropData = broccoli_cropData.merge(broccoli_data.drop(['lat', 'long', 'cropWeight'], axis=1).groupby(['id']).sum().
                                            add_suffix('_sum'), left_on='id', right_on='id')
broccoli_cropData.head()

In [ ]:
# Scatterplot Summe NDRE-85%-Quantil - Gewicht
sns.jointplot(x='NDRE_85_QUANTILE_sum', y='cropWeight', data=broccoli_cropData, kind='reg',
             joint_kws={'line_kws':{'color':'cyan'}})

# Horizontale Linie für minimales Erntegewicht
plt.axhline(y=300, color='red')
plt.show()

In [ ]:
# Scatterplot Summe Median-NDVI - Gewicht
sns.jointplot(x='medianNDVI_sum', y='cropWeight', data=broccoli_cropData, kind='reg',
             joint_kws={'line_kws':{'color':'cyan'}})

# Horizontale Linie für minimales Erntegewicht
plt.axhline(y=300, color='red')
plt.show()

In [ ]:
# Berechnung der Integrale des Median-NDVI, Median-NDRE
# x-Achse für Integration ist die Anzahl Tage seit erstem Messtag (Erster Messtag = 0) 
from scipy import integrate

# Hinzufügen der Spalten mit Initialwert 0.0
broccoli_cropData['medianNDVI_Integral'] = 0.0
broccoli_cropData['medianNDRE_Integral'] = 0.0
broccoli_cropData['pixelCount_Integral'] = 0.0

# x-Werte für Integration (Tage seit erstem Messdatum)
x = [0, 7, 13, 21, 28, 36, 47, 56, 62]

# Berechnung der Integrale für jeden Brokkoli
for index, row in broccoli_cropData.iterrows():
    #if(len(broccoli_data[broccoli_data.id==row['id']]['medianNDVI'].to_numpy()) == 9):
    broccoli_cropData.at[index, 'medianNDVI_Integral'] = integrate.cumtrapz(broccoli_data[broccoli_data.id==row['id']]
                                                                            ['medianNDVI'].to_numpy(), x=x, initial=0)[8]
    broccoli_cropData.at[index, 'medianNDRE_Integral'] = integrate.cumtrapz(broccoli_data[broccoli_data.id==row['id']]
                                                                            ['medianNDRE'].to_numpy(), x=x, initial=0)[8]
    broccoli_cropData.at[index, 'pixelCount_Integral'] = integrate.cumtrapz(broccoli_data[broccoli_data.id==row['id']]
                                                                            ['pixelCount'].to_numpy(), x=x, initial=0)[8]

broccoli_cropData.head()

In [ ]:
# Scatterplot Integral Median-NDVI - Gewicht
sns.jointplot(x='medianNDVI_Integral', y='cropWeight', data=broccoli_cropData, kind='reg',
             joint_kws={'line_kws':{'color':'cyan'}})

# Skala Integral NDVI auf [40,50] setzen
axes = plt.gca()
axes.set_xlim([40,50])

# Horizontale Line für minimales Reifegewicht
plt.axhline(y=300, color='red')
plt.show()

In [ ]:
# Scatterplot Integral Median-NDRE - Gewicht
sns.jointplot(x='medianNDRE_Integral', y='cropWeight', data=broccoli_cropData, kind='reg',
             joint_kws={'line_kws':{'color':'cyan'}})

# Skala Integral NDRE auf 6,16] setzen
axes = plt.gca()
axes.set_xlim([6,16])

# Horizontale Linie für minimales Reifegewicht
plt.axhline(y=300, color='red')
plt.show()

In [ ]:
# Verrechnung NDVI/NDRE Integrale
broccoli_cropData['NDRE_NDVI_norm'] = ((broccoli_cropData['medianNDRE_Integral'] * broccoli_cropData['medianNDVI_Integral'])
                                       / (broccoli_cropData['medianNDRE_Integral'] + broccoli_cropData['medianNDVI_Integral']))

# Scatterplot Verrechnung NDVI/NDRE Integrale - Gewicht
sns.jointplot(x='NDRE_NDVI_norm', y='cropWeight', data=broccoli_cropData, kind='reg',
             joint_kws={'line_kws':{'color':'cyan'}})

# Horizontale Linie für minimales Reifegewicht
plt.axhline(y=300, color='red')
plt.show()


In [ ]:
# Verrechnung NDVI / pixel Count
broccoli_cropData['NDVI_pixelCount_norm'] = ((broccoli_cropData['pixelCount_Integral'] * broccoli_cropData['medianNDVI_Integral'])
                                       / (broccoli_cropData['pixelCount_Integral'] + broccoli_cropData['medianNDVI_Integral']))

# Scatterplot Verrechnung NDVI/NDRE Integrale - Gewicht
sns.jointplot(x='NDVI_pixelCount_norm', y='cropWeight', data=broccoli_cropData, kind='reg',
             joint_kws={'line_kws':{'color':'cyan'}})

# Horizontale Linie für minimales Reifegewicht
plt.axhline(y=300, color='red')
plt.show()

#### Analyse unreifer, schadhafter Brokkoli
Nun werden noch die unreifen und schadhaften (d.h. eines der Ernte Labels wurde gesetzt) untersucht. Allenfalls können Ausreisser bei der Pixel-Anzahl/NDVI/NDRE auf einen schadhaften Zustand bei der Ernte hinweisen.<br>
Wie oben erwähnt sind die Labels:<br>
* Fehlt: Brokkoli wurde bei der Ernte nicht gefunden
* Überreif
* Verkümmert: Pflanze ist fast gar nicht gewachsen und hat kleine Blätter
* Kein Kopf: Pflanze hat zwar grosse Blätter aber keinen 'Kopf' bzw. keine Blüte
* Faul
* Unreif (alle Brokkoli unter 300g)

In [ ]:
# Scatterplot Verrechnung NDVI/NDRE Integrale - Erntelabel
columnsCropLabels = ['cropMissing', 'cropOverripe', 'cropRudimentary', 'cropRotten', 'cropUnripe', 'cropNoBlossom']
for column in columnsCropLabels:    
    sns.scatterplot(x='medianNDRE_Integral', y=column, data=broccoli_cropData)
    axes = plt.gca()
    axes.set_ylim([-0.1,1.1])    
    plt.show()

In [ ]:
for column in columnsCropLabels:    
    sns.scatterplot(x='medianNDVI_Integral', y=column, data=broccoli_cropData)
    axes = plt.gca()
    axes.set_ylim([-0.1,1.1])    
    plt.show()

In [ ]:
corr = broccoli_cropData.corr()
fig = plt.figure()
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111)
cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,len(broccoli_cropData.columns),1)
ax.set_xticks(ticks)
plt.xticks(rotation=90)
ax.set_yticks(ticks)
ax.set_xticklabels(broccoli_cropData.columns)
ax.set_yticklabels(broccoli_cropData.columns)
plt.show()

In [ ]:
corr['cropWeight']

In [ ]:
corr['cropUnripe']

In [ ]:
for column in columnsCropLabels:    
    sns.scatterplot(x='pixelCount_Integral', y=column, data=broccoli_cropData)
    axes = plt.gca()
    axes.set_ylim([-0.1,1.1])    
    plt.show()

In [ ]:
# Aufteilung der Daten in reife, unreife (Gewicht < 300g) Brokkoli
broccoli_data_lastDate_unripe = broccoli_data_lastDate[broccoli_data_lastDate.cropWeight < 300]
broccoli_data_lastDate_ripe = broccoli_data_lastDate[broccoli_data_lastDate.cropWeight >= 300]

print("Anzahl unreife Brokkoli (Gewicht < 300g): " + str(len(broccoli_data_lastDate_unripe)))
print("Durchschnitt meanNDVI unreife: " + str(broccoli_data_lastDate_unripe['meanNDVI'].mean()))
print("Durchschnitt meanNDRE unreife: " + str(broccoli_data_lastDate_unripe['meanNDRE'].mean()))
print("Durchschnitt medianNDRE unreife: " + str(broccoli_data_lastDate_unripe['medianNDRE'].mean()))
print("Durchschnitt NDRE_85_QUANTILE unreife: " + str(broccoli_data_lastDate_unripe['NDRE_85_QUANTILE'].mean()))
print("Durchschnitt NDRE_15_QUANTILE unreife: " + str(broccoli_data_lastDate_unripe['NDRE_15_QUANTILE'].mean()))
print()
print("Anzahl reife Brokkoli (Gewicht >= 300g): " + str(len(broccoli_data_lastDate_ripe)))
print("Durchschnitt meanNDVI reife: " + str(broccoli_data_lastDate_ripe['meanNDVI'].mean()))
print("Durchschnitt meanNDRE reife: " + str(broccoli_data_lastDate_ripe['meanNDRE'].mean()))
print("Durchschnitt medianNDRE reife: " + str(broccoli_data_lastDate_ripe['medianNDRE'].mean()))
print("Durchschnitt NDRE_85_QUANTILE reife: " + str(broccoli_data_lastDate_ripe['NDRE_85_QUANTILE'].mean()))
print("Durchschnitt NDRE_15_QUANTILE reife: " + str(broccoli_data_lastDate_ripe['NDRE_15_QUANTILE'].mean()))

## Meteomatics Daten

In [ ]:
# Abfrage definieren und ausführen: Alle Meteomatics-Daten
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
SQL_Query = pd.read_sql_query('SELECT * FROM dbo.meteomatics', cnxn)
meteomatics_data = pd.DataFrame(SQL_Query)
cnxn.close()

meteomatics_data.head()

In [ ]:
# Aggregieren der Meteomatics-Daten zwischen den Messzeitpunkten
# Datum eine Woche vor erstem Messdatum hinzufügen -> Wetterdaten-Aggregation bis zu diesem Zeitpunkt
i = 0
meteomatics_aggregates = pd.DataFrame()
dates_Meteomatics =['2019-04-11'] + dates
while i < (len(dates_Meteomatics) - 1):
    weekly_data = meteomatics_data[((meteomatics_data.timestamp > dates_Meteomatics[i]) & (meteomatics_data.timestamp < dates_Meteomatics[i+1]))][['temperature', 'precipitation', 'radiation', 'relativeHumidity', 'wind', 'cloudCover']]
    weekly_data_agg = pd.DataFrame(weekly_data.agg(['mean']))
    weekly_data_agg['dateFrom'] = dates_Meteomatics[i]
    weekly_data_agg['dateTo'] = dates_Meteomatics[i+1]
    weekly_data_agg['index'] = i
    weekly_data_agg.set_index(['index'], inplace=True)
    meteomatics_aggregates = pd.concat([meteomatics_aggregates, weekly_data_agg], axis=0)
    
    i = i + 1
    
display(meteomatics_aggregates)

In [ ]:
broccoli_data_aggregates = pd.DataFrame()
for date in dates:    
    broccoli_data_byDate = broccoli_data[broccoli_data.timestamp == date][['meanNDVI', 'NDRE_85_QUANTILE', 'pixelCount']]
    broccoli_data_byDate_agg = pd.DataFrame(broccoli_data_byDate.agg(['mean']))
    broccoli_data_byDate_agg = broccoli_data_byDate_agg.add_prefix('meanAgg_')
    broccoli_data_byDate_agg['date'] = date
    #weekly_data_agg.set_index(['index'], inplace=True)
    broccoli_data_aggregates = pd.concat([broccoli_data_aggregates, broccoli_data_byDate_agg], axis=0)
    
display(broccoli_data_aggregates)

In [ ]:
#plt.plot(broccoli_data_aggregates['date'],broccoli_data_aggregates[['meanNDVI']])
#plt.plot(meteomatics_aggregates['dateTo'],meteomatics_aggregates[['temperature', 'relativeHumidity','precipitation']])
#plt.show

meteomatics_aggregates.plot.line(x='dateTo', subplots=True, figsize=(10,8))
broccoli_data_aggregates.plot.line(x='date', subplots=True, figsize=(10,8))

In [ ]:
meteomatics_data_reindexed = meteomatics_data.copy()
meteomatics_data_reindexed['timestamp'] = pd.to_datetime(meteomatics_data_reindexed['timestamp'])
meteomatics_data_reindexed.set_index("timestamp", drop=True, inplace=True)

tmp_weather = meteomatics_data_reindexed.resample('D').median()
tmp_weather = tmp_weather.add_prefix('median_')

tmp_weather_sum = meteomatics_data_reindexed.resample('D').sum()
tmp_weather['sum_precipitation'] = tmp_weather_sum['precipitation']
tmp_weather['sum_accumulatedEnergy'] = tmp_weather_sum['accumulatedEnergy']

print(tmp_weather.shape)
tmp_weather.describe()

In [ ]:
broccoli_data_reindexed = broccoli_data.copy()
broccoli_data_reindexed['timestamp'] = pd.to_datetime(broccoli_data_reindexed['timestamp'])
broccoli_data_reindexed.set_index("timestamp", drop=True, inplace=True)

tmp_broccoli = broccoli_data_reindexed.resample('D').median()
tmp_broccoli.drop(['id', 'lat', 'long'], axis=1, inplace=True)
tmp_broccoli = tmp_broccoli.add_prefix('median_')
#tmp_broccoli.dropna(axis=0, inplace=True)

print(tmp_broccoli.shape)
tmp_broccoli.head()

In [ ]:
weather_broccoli_daily = pd.concat([tmp_weather, tmp_broccoli], axis=1)
weather_broccoli_daily = weather_broccoli_daily.interpolate(method='linear')
weather_broccoli_daily = weather_broccoli_daily.drop(columns=['median_hail','median_frostDepth'])
print(weather_broccoli_daily.shape)
weather_broccoli_daily

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(2,1,1)
plt.xlabel('meanNDVI vs. temperature')

ax1 = weather_broccoli_daily['median_meanNDVI'].plot(color='blue', grid=True, label='meanNDVI')
ax3 = weather_broccoli_daily['median_meanNDRE'].plot(color='green', grid=True, label='meanNDRE', sharey=ax1)

ax2 = weather_broccoli_daily['median_temperature'].plot(color='red', grid=True, secondary_y=True, label='temperature')
ax4 = weather_broccoli_daily['median_absoluteHumidity'].plot(color='orange', grid=True, secondary_y=True, label='absoluteHumidity',
                                                      sharey=ax2)
ax6 = weather_broccoli_daily['median_evaporation'].plot(color='brown', grid=True, label='evaporation',
                                                      sharey=ax2)

h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()

plt.legend(h1+h2, l1+l2, loc=2)

plt.subplot(2,1,2)
ax5 = weather_broccoli_daily['median_pixelCount'].plot(color='black', grid=True, secondary_y=False, label='pixelCount')

h5, l5 = ax5.get_legend_handles_labels()

plt.legend(h5, l5, loc=2)

plt.show()

In [ ]:
#weather_broccoli_daily.plot(x='median_pixelCount', y=['median_meanNDRE','median_meanNDVI'])

In [ ]:
corr = weather_broccoli_daily.corr()
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111)
cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,len(weather_broccoli_daily.columns),1)
ax.set_xticks(ticks)
plt.xticks(rotation=90)
ax.set_yticks(ticks)
ax.set_xticklabels(weather_broccoli_daily.columns)
ax.set_yticklabels(weather_broccoli_daily.columns)
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(2,1,1)
plt.xlabel('meanNDVI, meanNDRE vs. temperature')

ax1 = weather_broccoli_daily['median_meanNDVI'].plot(color='blue', grid=True, label='meanNDVI')

#ax2 = weather_broccoli_daily['relativeHumidity'].plot(color='red', grid=True, secondary_y=True, label='relativeHumidity', sharey=ax2)
#ax4 = weather_broccoli_daily['absoluteHumidity'].plot(color='orange', grid=True, secondary_y=True, label='absoluteHumidity', sharey=ax2)
#ax5 = weather_broccoli_daily['dewPoint'].plot(color='green', grid=True, secondary_y=True, label='dewPoint', sharey=ax2)
#ax6 = weather_broccoli_daily['radiation'].plot(color='violet', grid=True, secondary_y=True, label='radiation', sharey=ax2)
#ax7 = weather_broccoli_daily['growingDegreeDays'].plot(color='black', grid=True, secondary_y=False, label='growingDegreeDays', sharey=ax2)

h1, l1 = ax1.get_legend_handles_labels()
#h2, l2 = ax2.get_legend_handles_labels()

plt.legend(h1, l1, loc=2)

plt.subplot(2,1,2)
ax3 = weather_broccoli_daily['median_meanNDRE'].plot(color='green', grid=True, label='meanNDRE')
#ax8 = weather_broccoli_daily['relativeHumidity'].plot(color='red', grid=True, secondary_y=True, label='relativeHumidity')
#ax9 = weather_broccoli_daily['absoluteHumidity'].plot(color='orange', grid=True, secondary_y=True, label='absoluteHumidity', sharey=ax8)
#ax10 = weather_broccoli_daily['dewPoint'].plot(color='green', grid=True, secondary_y=True, label='dewPoint', sharey=ax8)
#ax11 = weather_broccoli_daily['radiation'].plot(color='violet', grid=True, secondary_y=True, label='radiation', sharey=ax8)
#ax12 = weather_broccoli_daily['growingDegreeDays'].plot(color='black', grid=True, secondary_y=True, label='growingDegreeDays',sharey=ax8)

h5, l5 = ax3.get_legend_handles_labels()
#h6, l6 = ax8.get_legend_handles_labels()

plt.legend(h5, l5, loc=2)

plt.show()